Image classification on CIFAR-10 image dataset using multi-layer perceptrons in python without using any API. 

In [1]:
import cPickle
import numpy as np

In [2]:
#unpickle the pickled data
def unpickle(file):
    with open(file, 'rb') as f:
        dict = cPickle.load(f)
    return dict

In [3]:
"""
Merge batches from CIFAR-10 pickled data
Input: num_to_load - number of batches of CIFAR-10 to load and merge
output: merged features and labels from specified no. of batches of CIFAR-10  
"""
def merge_batches(num_to_load=1):
    for i in range(num_to_load):
        path = "/Users/trinakarmakar/downloads/cifar-10-batches-py/"
        filename = "data_batch_" + str(i + 1)
        file = path+filename
        data = unpickle(file)
        if i == 0:
            features = data["data"]
            labels = np.array(data['labels'])
        else:
            features = np.append(features, data["data"], axis=0)
            labels = np.append(labels, data["labels"], axis=0)
    return features, labels

In [4]:
def one_hot_encode(data):
    """
    Encode target label IDs to one hot vecotr of size L where 
    L is the number of unique labels
    Params: List of label IDs
    Output: List of one hot vectors
    """
    one_hot = np.zeros((data.shape[0], 10))
    one_hot[np.arange(data.shape[0]), data] = 1
    return one_hot

In [5]:
def normalize(data):
    #Normalize the image pixels
    return data / 255.0

In [6]:
def preprocess(num_to_load=1):
    #Helper function to load and preprocess cifar-10 data
    X, y = merge_batches(num_to_load=1)
    X = normalize(X)
    X = X.reshape(-1, 3072, 1)
    y = one_hot_encode(y)
    y = y.reshape(-1,10,1)
    return X, y

In [7]:
def dataset_split(X, y, ratio=0.8):
    split = int(ratio * X.shape[0])
    indices = np.random.permutation(X.shape[0])
    training_idx, val_idx = indices[:split], indices[split:]
    X_train, X_val = X[training_idx, :], X[val_idx, :]
    y_train, y_val = y[training_idx, :], y[val_idx, :]
    print("Records in training dataset", X_train.shape[0])
    print("Records in validation dataset", X_val.shape[0])
    return X_train, y_train, X_val, y_val

In [8]:
def sigmoid(out):
    return 1.0 / (1.0 + np.exp(-out))

In [9]:
def delta_sigmoid(out):
    #Derivative of sigmoid activation
    return sigmoid(out) * (1 - sigmoid(out))

In [10]:
def sigmoidcrossentropyloss(a, y):
    #calculate sigmoid cross entropy loss
    return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

In [ ]:
class DNN(object):
    '''
    class to define neural network
    '''
    def __init__(self, sizes):
        '''
        initialize bias and weight using Gausian distribution with mean = 0 and variance = 1
        '''
        self.num_layers = len(sizes)
        #setting appropriate dimensions for weights and biases
        self.biases = [np.random.randn(y, 1) for y in sizes[1:]]
        self.weights = [np.random.randn(y,x) for x, y in zip(sizes[:-1], sizes[1:])]
    def feedforward(self, x):
        '''
        Forward passes an image feature matrix through the deep network
        params: x is image features
        '''
        activation = x
        activations = [x] #list to store activations of every layers
        outs = [] # list to store vectors for every layers
        for b, w in zip(self.biases, self.weights):
            out = np.dot(w, activation) + b
            outs.append(out)
            activation = sigmoid(out)
            activations.append(activation)
        return outs, activations
    def get_batch(self, X, y, batch_size):
        '''
        A data iterator for batching image features and labels
        '''
        for batch_idx in range(0, X.shape[0], batch_size):
            batch = zip(X[batch_idx:batch_idx+batch_size],
                       y[batch_idx:batch_idx + batch_size])
            yield batch
    def train(self, X, y, batch_size = 100, learning_rate = 0.2, epochs = 2):
        '''
        Train neural netwrok by batches of images. The weights and biases are updated through 
        backpropagation on batches using SGD. 
        Params: X, y = Image observation and labels
        batch_size =size of batch of input data
        learning_rate = eta; control the update rate of model parameters
        epochs = no. of times of iteration to train the model
        '''
        n_batches = X.shape[0] / batch_size
        for j in xrange(epochs):
            batch_iter = self.get_batch(X, y, batch_size)
            for i in range(n_batches):
                batch = batch_iter.next()
                del_b = [np.zeros(b.shape) for b in self.biases]
                del_w = [np.zeros(w.shape) for w in self.weights]
                for batch_X, batch_y in batch:
                    loss, delta_del_b, delta_del_w = self.backpropagation(
                         batch_X, batch_y)
                    del_b = [db + ddb for db, ddb in zip(del_b, delta_del_b)]
                    del_w = [dw + ddw for dw, ddw in zip(del_w, delta_del_w)]
                    self.weights = [w - (learning_rate / batch_size) * delw for w, delw in zip(self.weights, del_w)]
                    self.biases = [b - (learning_rate / batch_size) * delb for b, delb in zip(self.biases, del_b)]
                    print("\n Epoch %d complete\tLoss: %f\n"%(j, loss))
    def backpropagation(self, X, y):
        '''
        Using derivative gradient of cost function is calculated
        '''
        del_b = [np.zeros(b.shape) for b in self.biases]
        del_w = [np.zeros(w.shape) for w in self.weights]
        
        outs, activations = self.feedforward(X) # feedforward to compute output to calculate loss
        loss = sigmoidcrossentropyloss(activations[-1],y)
        delta_cost = activations[-1] - y # cost to be mininmized
        delta = delta_cost
        del_b[-1] = delta
        del_w[-1] = np.dot(delta, activations[-2].T)
        #propagate gradients backward 
        for k in xrange(2, self.num_layers):
            out = outs[-k]
            delta_activation = delta_sigmoid(out)
            delta = np.dot(self.weights[-k + 1].T, delta) * delta_activation
            del_b[-k] = delta
            del_w[-k] = np.dot(delta, activations[-k - 1].T)
        return (loss, del_b, del_w)
        
    def eval(self, X, y):
        #Predic the label and compare the model accuracy
        
        count = 0
        for x, _y in zip(X, y):
            outs, activations = self.feedforward(X)
            #position of maximum value is the predicted labels
            if np.argmax(activations[-1]) == np.argmax(_y):
                count += 1
        print("Accuracy: %f" %(float(count) / X.shape[0] * 100))
        
    def predict(self, X):
        
        labels = unpickle(path+"batches.meta")["label_names"]
        preds = np.array([])
        for x in X:
            outs, activations = self.feedforward(x)
            preds = np.append(preds, np.argmax(activations[-1]))
        preds = np.array([labels[int(p)] for p in preds])
        return preds
    
    
def main():
    X, y = preprocess(num_to_load=1)
    X_train, y_train, X_val, y_val = dataset_split(X, y)
    model = DNN([3072, 50, 30, 10]) # Initialize the model using 32*32*3 image 
    model.train(X_train, y_train, epochs=15)
    model.eval(X_val, y_val)
    test_X = unpickle(path+"test_batch")["data"] / 255.0
    test_X = test_X.reshape(-1, 3072, 1)
    print(model.predict(test_X))  
main()

('Records in training dataset', 8000)
('Records in validation dataset', 2000)

 Epoch 0 complete	Loss: 19.902027


 Epoch 0 complete	Loss: 10.134308


 Epoch 0 complete	Loss: 16.009127


 Epoch 0 complete	Loss: 9.231338


 Epoch 0 complete	Loss: 13.922058


 Epoch 0 complete	Loss: 9.848298


 Epoch 0 complete	Loss: 12.201828


 Epoch 0 complete	Loss: 13.371090


 Epoch 0 complete	Loss: 9.356876


 Epoch 0 complete	Loss: 6.030037


 Epoch 0 complete	Loss: 4.924358


 Epoch 0 complete	Loss: 6.238110


 Epoch 0 complete	Loss: 3.628627


 Epoch 0 complete	Loss: 6.410789


 Epoch 0 complete	Loss: 1.593625


 Epoch 0 complete	Loss: 7.929324


 Epoch 0 complete	Loss: 4.777761


 Epoch 0 complete	Loss: 4.239056


 Epoch 0 complete	Loss: 5.359840


 Epoch 0 complete	Loss: 2.591541


 Epoch 0 complete	Loss: 2.754243


 Epoch 0 complete	Loss: 4.396471


 Epoch 0 complete	Loss: 4.402387


 Epoch 0 complete	Loss: 5.313846


 Epoch 0 complete	Loss: 3.373975


 Epoch 0 complete	Loss: 4.311851


 Epoc